# Extraction of SDM
### Collect spin-density matrix elements from the `ROOT`-file and save them to a text file

In [1]:
import ROOT

Welcome to JupyROOT 6.14/00


In [2]:
name = '0.100000-0.112853'

In [44]:
############################################

def get_cve(myfile,name,i) :
    hist = myfile.Get(name)
    return [hist.GetBinCenter(i), hist.GetBinContent(i), hist.GetBinError(i)]

############################################

def get_SDM(myfile,bin) :
    SDM = [[0 for x in range(88)] for y in range(88)]
    eSDM = [[0 for x in range(88)] for y in range(88)]
    # zero epsilon
    for i in range(88) :
        v = get_cve(myfile,'h'+str(i+1), bin)
        SDM[i][i] = v[1]
        eSDM[i][i] = v[2]
    # positive epsilon
    for i in range(1,81) :
        for j in range(1,81) :
            if (i >= j) : continue
            name = 'h1'+'{:03d}'.format(i+1)+'{:03d}'.format(j+1)
            vr = get_cve(myfile,name, bin)
            name = 'h2'+'{:03d}'.format(i+1)+'{:03d}'.format(j+1)
            vi = get_cve(myfile,name, bin)
            SDM[i][j] = vr[1]+vi[1]*1j
            eSDM[i][j] = v[2]+vi[2]*1j
            SDM[j][i] = vr[1]-vi[1]*1j
            eSDM[j][i] = v[2]-vi[2]*1j
    # negative epsilon
    for i in range(82,88) :
        for j in range(82,88) :
            if (i >= j) : continue
            name = 'h1'+'{:03d}'.format(i+1)+'{:03d}'.format(j+1)
            vr = get_cve(myfile,name, bin)
            name = 'h2'+'{:03d}'.format(i+1)+'{:03d}'.format(j+1)
            vi = get_cve(myfile,name, bin)
            SDM[i][j]  = vr[1]+vi[1]*1j
            eSDM[i][j] = vr[2]+vi[2]*1j
            SDM[j][i]  = vr[1]-vi[1]*1j
            eSDM[j][i] = vr[2]-vi[2]*1j
    return SDM, eSDM

############################################

import numpy as np
def save2file(output, inmat) :
    mat=np.matrix([np.array(v).real for v in inmat])
    np.savetxt(output+'.re',mat)
    mat=np.matrix([np.array(v).imag for v in inmat])
    np.savetxt(output+'.im',mat)


Save a single SDM

In [38]:
myfile = ROOT.TFile('/home/mikhasenko/cernbox/tmp/pwa_results/hfit_'+names[2]+'.root')

SDM91, eSDM91 = get_SDM(myfile,91)
# save2file('sdm91', SDM91)
# save2file('sdm91-err', eSDM91)
print(SDM91[1][1])

myfile.Close()

526.094482422


Save all SDMs for the fiven t' in the file 

In [19]:
myfile = ROOT.TFile('/home/mikhasenko/cernbox/tmp/pwa_results/hfit_0.100000-0.112853.root')

for b in range(1,101) :
    SDM_b, eSDM_b = get_SDM(myfile,b)
    save2file('SDMs/'+name+'/'+'sdm'+str(b), SDM_b)
    save2file('SDMs/'+name+'/'+'sdm'+str(b)+'-err', eSDM_b)
    
myfile.Close()

Save all SDMs form all t'-slices in files 

In [45]:
def save_all_sdm(rootfilename, outfilename) :
    myfile = ROOT.TFile(rootfilename)
    Nb = 100
    for b in range(1,Nb+1) :
        SDM_b, eSDM_b = get_SDM(myfile,b)
        save2file(outfilename+'/'+'sdm'+str(b), SDM_b)
        save2file(outfilename+'/'+'sdm'+str(b)+'-err', eSDM_b)
    myfile.Close()

In [46]:
names = ["0.100000-0.112853", "0.112853-0.127471", "0.127471-0.144385", "0.144385-0.164401",
         "0.164401-0.188816", "0.188816-0.219907", "0.219907-0.262177", "0.262177-0.326380",
         "0.326380-0.448588", "0.448588-0.724294", "0.724294-1.000000"];

In [47]:
for name in names :
    print(name)
    save_all_sdm('/home/mikhasenko/cernbox/tmp/pwa_results/hfit_'+name+'.root',"SDMs/"+name)

0.100000-0.112853
0.112853-0.127471
0.127471-0.144385
0.144385-0.164401
0.164401-0.188816
0.188816-0.219907
0.219907-0.262177
0.262177-0.326380
0.326380-0.448588
0.448588-0.724294
0.724294-1.000000
